In [1]:
from IPython import get_ipython


In [2]:
# Insira seu project token aqui


 # MARATONA BEHIND THE CODE 2020

 ## DESAFIO 7 - TNT

 <hr>

 ## Installing Libs

In [3]:
get_ipython().system('pip install scikit-learn --upgrade')
get_ipython().system('pip install xgboost --upgrade')
get_ipython().system('pip install imblearn --upgrade')
get_ipython().system('pip install cloudant --upgrade')


Requirement already up-to-date: scikit-learn in c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages (0.23.2)
Requirement already up-to-date: xgboost in c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages (1.2.0)
Requirement already up-to-date: imblearn in c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages (0.0)
Requirement already up-to-date: cloudant in c:\users\lenovo\appdata\local\programs\python\python38\lib\site-packages (2.14.0)


 <hr>

 ## Download dos conjuntos de dados em formato .csv

In [ ]:
import pandas as pd



In [ ]:
# Insira aqui o pandasDataFrame.
from cloudant import Cloudant
#  from cloudant.client import Cloudant
u = '9aa6ed7a-5e4b-4630-b049-4b9994d9365b-bluemix'
p = 'd05f4290760adade9c4e3977bbabab948bd5551ee735877e2492f800360f5258'
a = '9aa6ed7a-5e4b-4630-b049-4b9994d9365b-bluemix'
client = Cloudant(u, p, account=a, connect=True, auto_renew=True)
db = client['desafio7_iot_dataset']



In [ ]:
docs = []
response = db.all_docs(limit=500, include_docs=True)
for r in response['rows']:
    docs.append(r['doc'])
type(docs)
df = pd.DataFrame(data=docs)
df['LAT'] = df['LAT'].astype(float)
df['LONG'] = df['LONG'].astype(float)
df['Tempo'] = pd.to_datetime(df['Tempo'],errors='coerce')
df['year'] = df['Tempo'].dt.year
df['month'] = df['Tempo'].dt.month
df['day'] = df['Tempo'].dt.day
df['weekday'] = df['Tempo'].dt.weekday
df['week'] = df['Tempo'].dt.week
# trash = ['year','month','day','weekday','week']
trash = []
remove_trash = ['_id', '_rev','row','Tempo']
trash = trash + remove_trash
# dummies_column = ['Estação','year','month','day','weekday','week','LAT','LONG']
dummies_column = ['Estação','year','month','day','weekday','week','LAT','LONG']

df = df.drop(columns=[trash], inplace=False)
df.head()
# df.info()



In [ ]:
df.drop_duplicates(subset=None, inplace=True)
df.info()



In [ ]:
df_data_1 = df
df_training_dataset = df_data_1
df_training_dataset.tail()


 Sobre o arquivo "training_dataset.csv", temos algumas informações gerais sobre os pontos de vendas da TNT:

 **Tempo**

 **Estação**

 **LAT**

 **LONG**

 **Movimentação**

 **Original_473**

 **Original_269**

 **Zero**

 **Maçã-Verde**

 **Tangerina**

 **Citrus**

 **Açaí-Guaraná**

 **Pêssego**

 **TARGET**

In [ ]:
df_training_dataset.info()



In [ ]:
df_training_dataset.nunique()


 <hr>

 ## Detalhamento do desafio: classificação binária

 Este é um desafio cujo objetivo de negócio é a segmentação dos usuários de aplicativo de um banco. Para tal, podemos utilizar duas abordagens: aprendizado de máquina supervisionado (classificação) ou não-supervisionado (clustering). Neste desafio será aplicada a classificação, pois é disponível um dataset já com "labels", ou em outras palavras, já com exemplos de dados juntamente com a variável alvo.

 Na biblioteca scikit-learn temos diversos algoritmos para classificação. O participante é livre para utilizar o framework que desejar para completar esse desafio.

 Neste notebook será mostrado um exeplo de uso do algoritmo "Decision Tree" para classificar parte dos estudantes em seis diferentes perfís.

 # Atenção!

 A coluna-alvo neste desafio é a coluna ``TARGET``

 <hr>

 ## Pre-processando o dataset antes do treinamento

 ### Processando valores NaN com o SimpleImputer do sklearn

 Para os valores NaN, usaremos a substituição pela constante 0 como **exemplo**.

 Você pode escolher a estratégia que achar melhor para tratar os valores nulos :)

 Docs: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html?highlight=simpleimputer#sklearn.impute.SimpleImputer

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np


# impute_zeros = SimpleImputer(
#     missing_values=np.nan,
#     strategy='constant',
#     fill_value=0,
#     verbose=0,
#     copy=True
# )
def Missing_Value_imputer():
    imputer = SimpleImputer(
        missing_values=np.nan
        , strategy='most_frequent'
        )
    return imputer

impute_zeros = Missing_Value_imputer()



In [ ]:
# Exibindo os dados ausentes do conjunto de dados antes da primeira transformação (df)
print("Valores nulos no df_training_dataset antes da transformação SimpleImputer: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

# Aplicando a transformação ``SimpleImputer`` no conjunto de dados base
impute_zeros.fit(X=df_training_dataset)

# Reconstruindo um Pandas DataFrame com os resultados
df_training_dataset_imputed = pd.DataFrame.from_records(
    data=impute_zeros.transform(
        X=df_training_dataset
    ),
    columns=df_training_dataset.columns
)

# Exibindo os dados ausentes do conjunto de dados após a primeira transformação (df)
print("Valores nulos no df_training_dataset após a transformação SimpleImputer: \n\n{}\n".format(df_training_dataset_imputed.isnull().sum(axis = 0)))


 ### Eliminando colunas indesejadas

 Vamos **demonstrar** abaixo como usar o método **DataFrame.drop()**.

 Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [ ]:
df_training_dataset_imputed.tail()



In [ ]:
# df_training_dataset_rmcolumns = df_training_dataset_imputed.drop(columns=['Tempo', 'Estação', 'LAT', 'LONG', 'Movimentação'], inplace=False)
df_training_dataset_rmcolumns = df_training_dataset_imputed



In [ ]:
df_training_dataset_rmcolumns.tail()


 # Atenção!

 As colunas removidas acima são apenas para fim de exemplo, você pode usar as colunas que quiser e inclusive criar novas colunas com dados que achar importantes!


 ### Tratamento de de variáveis categóricas

 Como mencionado antes, os computadores não são bons com variáveis "categóricas" (ou strings).

 Dado uma coluna com variável categórica, o que podemos realizar é a codificação dessa coluna em múltiplas colunas contendo variáveis binárias. Esse processo é chamado de "one-hot-encoding" ou "dummy encoding". Se você não é familiarizado com esses termos, você pode pesquisar mais sobre isso na internet :)

In [ ]:
# Tratando variáveis categóricas com o método Pandas ``get_dummies()''
# df_training = pd.get_dummies(df_training_dataset_rmcolumns, columns=['Variável a ser aplicado método getDumies()'])
df_training = df_training_dataset_rmcolumns
df_training.tail()


 # Atenção!

 A coluna **TARGET** deve ser mantida como uma string. Você não precisa processar/codificar a variável-alvo.

 <hr>

 ## Treinando um classificador com base em uma árvore de decisão

 ### Selecionando FEATURES e definindo a variável TARGET

In [ ]:
def cat_columns(df_x):
    # Categorical boolean mask
    categorical_feature_mask = df_x.dtypes==object# filter categorical columns using mask and turn it into a list
    categorical_cols = df_x.columns[categorical_feature_mask].tolist()
    return categorical_cols

feature_columns = cat_columns(df_training_dataset_rmcolumns)
feature_columns.pop(feature_columns.index('TARGET'))
feature_columns
# Tratando variáveis categóricas com o método Pandas ``get_dummies()''
df_training = pd.get_dummies(df_training_dataset_rmcolumns, columns=dummies_column)
df_training.tail()



In [ ]:
df_training.columns
df_training.columns
feature_column2 = df_training.columns.tolist()
feature_column2.pop(feature_column2.index('TARGET'))
feature_column2



In [ ]:
df_training.columns



In [ ]:
features = df_training[feature_column2]
target = df_training['TARGET']  ## NÃO TROQUE O NOME DA VARIÁVEL TARGET.


 ### Dividindo nosso conjunto de dados em conjuntos de treinamento e teste

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=133)

    # %% [markdown]
    # ### Treinando uma árvore de decisão

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin

#=================Scaler
scaler = [
StandardScaler(),
MinMaxScaler(),
MaxAbsScaler(),
RobustScaler(quantile_range=(25, 75)),
PowerTransformer(method='yeo-johnson'),
# PowerTransformer(method='box-cox'),
QuantileTransformer(output_distribution='normal'),
QuantileTransformer(output_distribution='uniform'),
Normalizer()
]


#=================Classifier
classifier_test = [
OneVsRestClassifier(SVC())
,DecisionTreeClassifier(max_depth=5)
,SVC()
,SVC(kernel="linear", C=0.025)
,LogisticRegressionCV(cv=5, random_state=0)
,GradientBoostingClassifier(random_state=0)
,BaggingClassifier(base_estimator=SVC(),n_estimators=10, random_state=0).fit(features, target)
,ExtraTreesClassifier(n_estimators=100, random_state=0)
,HistGradientBoostingClassifier()
,MLPClassifier(random_state=1, max_iter=300)
,OneVsOneClassifier(LinearSVC(random_state=0))
,OutputCodeClassifier(estimator=RandomForestClassifier(random_state=0),random_state=0)
]
print('Importacao OK')

In [ ]:
def teste(model_scaler,model_classifier,name_scaler,name_classifier,model):
    # Método para criar um árvore de decisão
    from sklearn.tree import DecisionTreeClassifier


    # dtc = DecisionTreeClassifier(max_depth=15).fit(X_train, y_train)
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.pipeline import Pipeline
    from sklearn.ensemble import RandomForestClassifier

    # model_scaler = StandardScaler()
    # model_classifier = GradientBoostingClassifier(random_state=0)
    # model_classifier = RandomForestClassifier()

    classifier_model = Pipeline([('scaler', model_scaler), ('svc', model_classifier)])
    dtc = classifier_model
    print(dtc)
    dtc.fit(
        X_train,
        y_train
    )

    # %% [markdown]
    # ### Fazendo previsões na amostra de teste
    y_pred = dtc.predict(X_test)
    print(y_pred)

    # %% [markdown]
    # ### Analisando a qualidade do modelo através da matriz de confusão

    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
        accuracy = np.trace(cm) / float(np.sum(cm))
        misclass = 1 - accuracy
        if cmap is None:
            cmap = plt.get_cmap('Blues')
        plt.figure(figsize=(8, 6))
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        if target_names is not None:
            tick_marks = np.arange(len(target_names))
            plt.xticks(tick_marks, target_names, rotation=45)
            plt.yticks(tick_marks, target_names)
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        thresh = cm.max() / 1.5 if normalize else cm.max() / 2
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            if normalize:
                plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                        horizontalalignment="center",
                        color="white" if cm[i, j] > thresh else "black")
            else:
                plt.text(j, i, "{:,}".format(cm[i, j]),
                        horizontalalignment="center",
                        color="white" if cm[i, j] > thresh else "black")
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
        plt.show()

        cm = confusion_matrix(y_test, y_pred)
        accuracy = np.trace(cm) / float(np.sum(cm))
        end = f'accuracy: {accuracy:.4f} | S: {name_scaler} C: {name_classifier}'
        print(end)
        dict_test = {}
        if accuracy > 0.78:
            dict_test[f'M-{model}'] = end


        import json
        file_name = 'test_result'
        with open(f"{file_name}.json","a") as file:
            json.dump(dict_test, file)

        # with open(f"{file_name}.json", "w+") as file:
        #     data = json.load(file)
        #     data.update(dict_test)
        #     # file.seek(0)
        #     json.dump(data, file)

    from sklearn.metrics import confusion_matrix


    plot_confusion_matrix(confusion_matrix(y_test, y_pred), ['NORMAL', 'REABASTECER'])



In [ ]:
model_scaler = StandardScaler()
model_classifier = GradientBoostingClassifier(random_state=0)
teste(model_scaler,model_classifier,-1,-1,-2)


In [ ]:
count = 0
for i in range(len(scaler)):
        scaler_i = scaler[i]
        for j in range(len(classifier_test)):
            count += 1
            print(f'Model: {count}')
            classifier_test_j = classifier_test[j]
            teste(scaler_i,classifier_test_j,i,j,count)